In [ ]:
#pip install boto3

In [1]:
import boto3
import pandas as pd
import numpy as np

In [2]:
client = boto3.client('s3')

In [3]:
path = 's3://mcsp-hiring-external/test_input_file.csv'

In [ ]:
#pip install s3fs

In [4]:
df = pd.read_csv(path)
df.head()

,id,name,date_of_birth,Gender,device_type_us_state
0,0039c5a9-505a-4ece-80ac-6ee9225f0d2a,Liam,16/07/1978,NaN,android_CT-Connecticut
1,004d3491-8ff2-469e-8946-6fcf2e311329,Noah,04/03/2004,Female,iphone_DE-Delaware
2,00a9842b-7d31-45a1-b295-99d278588e40,William,10/09/1979,NaN,iphone_RI-Rhode Island
3,010acae5-8749-4cac-b5a9-ae83825b57b1,James,06/03/2012,F,iphone_PA-Pennsylvania
4,013a1028-4fd5-413a-b703-56fc15b773be,Oliver,23/06/1985,,android_CT-Connecticut


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    500 non-null    object
 1   name                  499 non-null    object
 2   date_of_birth         500 non-null    object
 3   Gender                387 non-null    object
 4   device_type_us_state  500 non-null    object
dtypes: object(5)
memory usage: 19.7+ KB


In [6]:
#Check Duplicate Values
df.duplicated().sum()

0

In [ ]:
#Check Missing Values
#print ("\nMissing values :  ", df.isnull().sum().values.sum())

In [7]:
#dataset with null value
df.isnull().sum()

id                        0
name                      1
date_of_birth             0
Gender                  113
device_type_us_state      0
dtype: int64

In [8]:
df = df.apply(lambda x: x.str.strip()).replace('', np.nan)
df.head()

,id,name,date_of_birth,Gender,device_type_us_state
0,0039c5a9-505a-4ece-80ac-6ee9225f0d2a,Liam,16/07/1978,NaN,android_CT-Connecticut
1,004d3491-8ff2-469e-8946-6fcf2e311329,Noah,04/03/2004,Female,iphone_DE-Delaware
2,00a9842b-7d31-45a1-b295-99d278588e40,William,10/09/1979,NaN,iphone_RI-Rhode Island
3,010acae5-8749-4cac-b5a9-ae83825b57b1,James,06/03/2012,F,iphone_PA-Pennsylvania
4,013a1028-4fd5-413a-b703-56fc15b773be,Oliver,23/06/1985,NaN,android_CT-Connecticut


In [9]:
df = df.dropna(axis = 0)

In [10]:
# there is no null values
df.isnull().sum()

id                      0
name                    0
date_of_birth           0
Gender                  0
device_type_us_state    0
dtype: int64

In [11]:
df.shape

(337, 5)

In [12]:
from datetime import datetime

In [13]:
# Make sure to parse all datetime columns in advance
df['date_of_birth'] = pd.to_datetime(df['date_of_birth'], errors='coerce')

In [14]:
now = pd.to_datetime('now')
now

Timestamp('2020-12-13 02:20:56.421277')

In [15]:
(now - df['date_of_birth']).astype('<m8[Y]')

1      16.0
3       8.0
5      28.0
8      23.0
9      40.0
       ... 
492    41.0
493    44.0
494    11.0
497    17.0
499    17.0
Name: date_of_birth, Length: 337, dtype: float64

In [16]:
age = (now.year - df['date_of_birth'].dt.year) - ((now.month - df['date_of_birth'].dt.month) < 0)
age

1      16
3       8
5      28
8      23
9      40
       ..
492    41
493    44
494    11
497    17
499    17
Name: date_of_birth, Length: 337, dtype: int64

In [17]:
# add age column from new data frame 
df['Age'] = age 
df

,id,name,date_of_birth,Gender,device_type_us_state,Age
1,004d3491-8ff2-469e-8946-6fcf2e311329,Noah,2004-04-03,Female,iphone_DE-Delaware,16
3,010acae5-8749-4cac-b5a9-ae83825b57b1,James,2012-06-03,F,iphone_PA-Pennsylvania,8
5,013f182f-42ac-4063-a774-1b0218c52fd8,Benjamin,1992-04-03,FEMALE,iphone_DE-Delaware,28
8,021c45e1-f551-4da8-885d-ad83fe102a15,Mason,1997-06-19,M,android_PA-Pennsylvania,23
9,025219f6-f3aa-4d1d-8dc3-b485ca5a4641,Logan,1980-04-12,Female,android_NJ-New Jersey,40
...,...,...,...,...,...,...
492,58b2ba9d-2319-4b50-97b0-80e73c481ee4,Uriel,1979-11-05,Male,iphone_PA-Pennsylvania,41
493,592ae939-bd3f-4e90-9347-0fe7817f3274,Ronald,1976-03-25,M,iphone_NH-New Hampshire,44
494,593769d9-d33f-44e9-a381-873963486972,Luciano,2009-11-04,FEMALE,iphone_DE-Delaware,11
497,59689cfb-27d9-4e95-81fd-c79511947df5,Kieran,2003-10-05,Female,android_CT-Connecticut,17


In [18]:
# new data frame with split value columns 
new_df = df["device_type_us_state"].str.split("_", n = 1, expand = True) 
new_df.head()

,0,1
1,iphone,DE-Delaware
3,iphone,PA-Pennsylvania
5,iphone,DE-Delaware
8,android,PA-Pennsylvania
9,android,NJ-New Jersey


In [19]:
# making separate device_type column from new data frame 
df["device_type"]= new_df[0] 

# making separate state_name column from new data frame 
df["state_name"]= new_df[1] 

# Dropping old Name columns 
df.drop(columns =["device_type_us_state"], inplace = True) 

# df display 
df

,id,name,date_of_birth,Gender,Age,device_type,state_name
1,004d3491-8ff2-469e-8946-6fcf2e311329,Noah,2004-04-03,Female,16,iphone,DE-Delaware
3,010acae5-8749-4cac-b5a9-ae83825b57b1,James,2012-06-03,F,8,iphone,PA-Pennsylvania
5,013f182f-42ac-4063-a774-1b0218c52fd8,Benjamin,1992-04-03,FEMALE,28,iphone,DE-Delaware
8,021c45e1-f551-4da8-885d-ad83fe102a15,Mason,1997-06-19,M,23,android,PA-Pennsylvania
9,025219f6-f3aa-4d1d-8dc3-b485ca5a4641,Logan,1980-04-12,Female,40,android,NJ-New Jersey
...,...,...,...,...,...,...,...
492,58b2ba9d-2319-4b50-97b0-80e73c481ee4,Uriel,1979-11-05,Male,41,iphone,PA-Pennsylvania
493,592ae939-bd3f-4e90-9347-0fe7817f3274,Ronald,1976-03-25,M,44,iphone,NH-New Hampshire
494,593769d9-d33f-44e9-a381-873963486972,Luciano,2009-11-04,FEMALE,11,iphone,DE-Delaware
497,59689cfb-27d9-4e95-81fd-c79511947df5,Kieran,2003-10-05,Female,17,android,CT-Connecticut


In [20]:
df['Gender'] = df.Gender.str.lower()
df.head()

,id,name,date_of_birth,Gender,Age,device_type,state_name
1,004d3491-8ff2-469e-8946-6fcf2e311329,Noah,2004-04-03,female,16,iphone,DE-Delaware
3,010acae5-8749-4cac-b5a9-ae83825b57b1,James,2012-06-03,f,8,iphone,PA-Pennsylvania
5,013f182f-42ac-4063-a774-1b0218c52fd8,Benjamin,1992-04-03,female,28,iphone,DE-Delaware
8,021c45e1-f551-4da8-885d-ad83fe102a15,Mason,1997-06-19,m,23,android,PA-Pennsylvania
9,025219f6-f3aa-4d1d-8dc3-b485ca5a4641,Logan,1980-04-12,female,40,android,NJ-New Jersey


In [21]:
# replacing f from female and m from male
df1 = (df.replace("f", "female"))

In [22]:
df2 = (df1.replace("m", "male"))
df2.head()

,id,name,date_of_birth,Gender,Age,device_type,state_name
1,004d3491-8ff2-469e-8946-6fcf2e311329,Noah,2004-04-03,female,16,iphone,DE-Delaware
3,010acae5-8749-4cac-b5a9-ae83825b57b1,James,2012-06-03,female,8,iphone,PA-Pennsylvania
5,013f182f-42ac-4063-a774-1b0218c52fd8,Benjamin,1992-04-03,female,28,iphone,DE-Delaware
8,021c45e1-f551-4da8-885d-ad83fe102a15,Mason,1997-06-19,male,23,android,PA-Pennsylvania
9,025219f6-f3aa-4d1d-8dc3-b485ca5a4641,Logan,1980-04-12,female,40,android,NJ-New Jersey


In [23]:
final_df = df2.copy()

In [24]:
final_df.groupby(['id','device_type','Age','Gender','state_name']).size()

id                                    device_type  Age  Gender  state_name      
004d3491-8ff2-469e-8946-6fcf2e311329  iphone       16   female  DE-Delaware         1
010acae5-8749-4cac-b5a9-ae83825b57b1  iphone       8    female  PA-Pennsylvania     1
013f182f-42ac-4063-a774-1b0218c52fd8  iphone       28   female  DE-Delaware         1
021c45e1-f551-4da8-885d-ad83fe102a15  android      23   male    PA-Pennsylvania     1
025219f6-f3aa-4d1d-8dc3-b485ca5a4641  android      40   female  NJ-New Jersey       1
                                                                                   ..
58b2ba9d-2319-4b50-97b0-80e73c481ee4  iphone       41   male    PA-Pennsylvania     1
592ae939-bd3f-4e90-9347-0fe7817f3274  iphone       44   male    NH-New Hampshire    1
593769d9-d33f-44e9-a381-873963486972  iphone       11   female  DE-Delaware         1
59689cfb-27d9-4e95-81fd-c79511947df5  android      17   female  CT-Connecticut      1
59a19d68-282a-4f27-ae03-016888016bbe  iphone       17   fem

In [25]:
# Finding how many days of data each store have
final_df.groupby('device_type')['device_type'].count()

device_type
android    166
iphone     171
Name: device_type, dtype: int64

In [27]:
# Get the first entry for each age
final_df.groupby('Age').first().head()

,id,name,date_of_birth,Gender,device_type,state_name
Age,,,,,,
4,2798f8b7-4c0b-4c9f-bb2b-113f928f588d,Marcus,2016-02-13,female,android,NJ-New Jersey
5,04a00d8c-7cf8-4d59-b17f-bb99313f3612,Owen,2015-07-01,male,iphone,MD-Maryland
6,057e35f3-f552-4d87-a375-e2f83b7d126f,Grayson,2014-06-28,male,android,DC-District of Columbia
7,222ef65c-7db7-4212-8fb7-44b44de8b793,Hayden,2013-07-28,male,iphone,VT-Vermont
8,010acae5-8749-4cac-b5a9-ae83825b57b1,James,2012-06-03,female,iphone,PA-Pennsylvania


In [28]:
final_df1 = final_df.groupby(['Age','device_type','Gender','state_name'])['Age'].sum()

In [30]:
final_df1.head(50)

Age  device_type  Gender  state_name             
4    android      female  MA-Massachusetts            4
                          ME-Maine                    4
                          NJ-New Jersey               4
                          PA-Pennsylvania             4
     iphone       female  VT-Vermont                  4
                  male    NH-New Hampshire           12
                          NJ-New Jersey               8
5    android      male    DE-Delaware                 5
                          NJ-New Jersey               5
     iphone       female  DE-Delaware                 5
                          MD-Maryland                 5
                          VT-Vermont                  5
                  male    CT-Connecticut              5
                          MD-Maryland                 5
                          RI-Rhode Island             5
6    android      female  CT-Connecticut              6
                          DE-Delaware                 

In [59]:
age

1      16
3       8
5      28
8      23
9      40
       ..
492    41
493    44
494    11
497    17
499    17
Name: date_of_birth, Length: 337, dtype: int64

In [71]:
# data found in this below 18 age group
age_range1 = final_df[(age <= 18)]
age_range1.head() 

,id,name,date_of_birth,Gender,Age,device_type,state_name
1,004d3491-8ff2-469e-8946-6fcf2e311329,Noah,2004-04-03,female,16,iphone,DE-Delaware
3,010acae5-8749-4cac-b5a9-ae83825b57b1,James,2012-06-03,female,8,iphone,PA-Pennsylvania
10,0260ecc0-df5c-433c-9cb9-84c8f96be2da,Alexander,2004-09-06,female,16,android,NH-New Hampshire
11,0292ddd9-3dbf-427c-b124-8c59a0e7e67e,Ethan,2006-02-07,female,14,iphone,RI-Rhode Island
24,04a00d8c-7cf8-4d59-b17f-bb99313f3612,Owen,2015-07-01,male,5,iphone,MD-Maryland


In [70]:
# data found in this more than equal to 19 and less than equal to 24 age group
age_range2 = final_df[(age >= 19) & (age <= 24)]
age_range2.head() 

,id,name,date_of_birth,Gender,Age,device_type,state_name
8,021c45e1-f551-4da8-885d-ad83fe102a15,Mason,1997-06-19,male,23,android,PA-Pennsylvania
22,04862122-fbae-4af1-8980-e82b5de65b3b,Joseph,1997-12-02,female,23,iphone,DE-Delaware
36,064042cf-dd6c-4e3e-9be6-4d4b736c77e3,Mateo,2000-04-28,female,20,iphone,PA-Pennsylvania
54,0a568bcb-432c-4db7-ba96-28d8f2b075ab,Christian,1997-10-12,male,23,iphone,DE-Delaware
75,0f376986-83a6-4b9e-b44d-d28af974b0a2,Jaxson,1998-08-29,male,22,android,RI-Rhode Island


In [73]:
# data found in this more than and equal to 25 and less than equal to 34 age group
age_range3 = final_df[(age >= 25) & (age <= 34)]
age_range3.head() 

,id,name,date_of_birth,Gender,Age,device_type,state_name
5,013f182f-42ac-4063-a774-1b0218c52fd8,Benjamin,1992-04-03,female,28,iphone,DE-Delaware
13,02b791f0-605d-46ac-9c8b-51a0caef1eed,Michael,1986-02-05,female,34,android,ME-Maine
18,03e34ddb-a0b0-4d13-b56d-6fd68e993039,Aiden,1991-12-01,female,29,android,DC-District of Columbia
20,04749bdb-34ed-4606-b70b-5ed79b79b9f1,Samuel,1993-05-27,male,27,android,CT-Connecticut
23,0487723b-29e3-4a1f-8041-819e557e85a0,Carter,1991-08-02,female,29,android,DC-District of Columbia


In [74]:
# data found in this more than and equal to 35 and less than equal to 44 age group
age_range4 = final_df[(age >= 35) & (age <= 44)]
age_range4.head() 

,id,name,date_of_birth,Gender,Age,device_type,state_name
9,025219f6-f3aa-4d1d-8dc3-b485ca5a4641,Logan,1980-04-12,female,40,android,NJ-New Jersey
14,02d4ba77-a4b4-420a-9442-15294ca2f707,Daniel,1979-11-04,male,41,android,PA-Pennsylvania
17,03d94086-1d9a-4fc4-83d4-12c631a197a8,Sebastian,1983-09-14,male,37,android,PA-Pennsylvania
51,09f04694-1b6b-4340-a216-1e85d6fca155,Charles,1977-08-17,female,43,android,PA-Pennsylvania
58,0b2e4a76-6fc3-40ee-99b5-6ebb63684f01,Ezra,1979-07-10,male,41,android,CT-Connecticut


In [75]:
# data found in this more than and equal to 45 and less than equal to 54 age group
age_range4 = final_df[(age >= 45) & (age <= 54)]
age_range4.head() 

,id,name,date_of_birth,Gender,Age,device_type,state_name
38,077edaa6-c799-40ee-9319-d1c8688117c9,Jaxon,1970-04-29,female,50,iphone,CT-Connecticut
63,0d054183-b441-4ee6-8d33-7de65f334470,Nolan,1974-11-24,male,46,android,CT-Connecticut
66,0d2f45be-e3df-4c71-ae2b-3af31b4ca340,Elias,1974-01-03,female,46,iphone,CT-Connecticut
76,0f4afc5d-0265-46af-b965-727f12a1d97a,Greyson,1974-10-12,female,46,android,MD-Maryland
87,113a39bd-1f23-4dbc-8881-94972da5cede,Xavier,1970-04-19,female,50,iphone,ME-Maine


In [77]:
# no data found in this +65 age group
age_range5 = final_df[(age >= 55) & (age <= 64)]
age_range5

,id,name,date_of_birth,Gender,Age,device_type,state_name


In [78]:
# no data found in this +65 age group
age_range6 = final_df[(age >= 65)]
age_range6

,id,name,date_of_birth,Gender,Age,device_type,state_name


In [80]:
# device has been used below 18 age group
age_range1.groupby('device_type')['device_type'].count()

device_type
android    54
iphone     60
Name: device_type, dtype: int64

In [81]:
# device has been used 19-24 age group
age_range2.groupby('device_type')['device_type'].count()

device_type
android    20
iphone     17
Name: device_type, dtype: int64

In [82]:
# device has been used 25-34 age group
age_range3.groupby('device_type')['device_type'].count()

device_type
android    35
iphone     32
Name: device_type, dtype: int64

In [83]:
# device has been used 35-44 age group
age_range4.groupby('device_type')['device_type'].count()

device_type
android    15
iphone     26
Name: device_type, dtype: int64

In [ ]:
# No data found more than age 45 age group

## <center> ------THANK YOU------ 